In [40]:
# Answers for part 3

import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact

In [41]:
def plot_gradient(V, F, R, p  = None):
    # Highlight in red the constrained faces
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)
    if p == None:
        p = mp.plot(V, F)
    
    p.add_lines(B, B + R * avg)
    
    return p

In [42]:
# Harmonic parametrization
v, f  = igl.read_triangle_mesh("./data/camel_head.off")
## Find the open boundary
bnd = igl.boundary_loop(f)

## Map the boundary to a circle, preserving edge proportions
bnd_uv = igl.map_vertices_to_circle(v, bnd)
## Harmonic parametrization for the internal vertices
uv_harmonic = igl.harmonic_weights(v, f, bnd, bnd_uv, 1)

p = subplot(v, f, uv = uv_harmonic, s=[1, 2, 0])
subplot(uv_harmonic, f, uv = uv_harmonic, shading={"wireframe": True, "flat": False}, s=[1, 2, 1], data=p, c = uv[:,0])


In [43]:
G = igl.grad(v, f)
gt = G @ uv_harmonic[:,0]
gt_vec_harmonic = np.hstack((gt[0:len(f):,None], np.hstack((gt[len(f):2*len(f):,None], gt[2*len(f)::,None]))))

p = plot(v, f, c = uv_harmonic[:,0], shading={"wireframe": False, "flat": False})
plot_gradient(v, f, gt_vec_harmonic, p)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

In [44]:
# LSCM parametrization

# Fix two points on the boundary
## Find the open boundary
bnd = igl.boundary_loop(f)
b = np.array([2, 1])
b[0] = bnd[0]
b[1] = bnd[int(bnd.size / 2)]
# b = igl.map_vertices_to_circle(v, bnd)

bc = np.array([[0.0, 0.0], [1.0, 0.0]])

# LSCM parametrization
_, uv_lscm = igl.lscm(v, f, b, bc)

p = subplot(v, f, uv=uv_lscm, shading={"wireframe": False, "flat": False}, s=[1, 2, 0])
subplot(uv_lscm, f, uv=uv_lscm, shading={"wireframe": True, "flat": False}, s=[1, 2, 1], data=p)


In [45]:
G = igl.grad(v, f)
gt = G @ uv_lscm[:,0]
gt_vec_lscm = np.hstack((gt[0:len(f):,None], np.hstack((gt[len(f):2*len(f):,None], gt[2*len(f)::,None]))))

p = plot(v, f, c = uv_lscm[:,0], shading={"wireframe": False, "flat": False})
plot_gradient(v, f, gt_vec_lscm, p)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…